In [20]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Para estandarizar características
from tensorflow.keras.models import Sequential  # Para inicializar modelos secuenciales
from tensorflow.keras.layers import Dense, Input  # Para definir capas densamente conectadas en la red neuronal
import joblib  # Para guardar y cargar modelos entrenados
import numpy as np  # Para cálculos numéricos
from sklearn.metrics import mean_squared_error, r2_score  # Para calcular métricas de rendimiento del modelo

In [21]:
# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

#Impresion de las columnas existentes 
print(df.columns)



Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo'], dtype='object')


In [22]:
# Dividir los datos en características (X) y el objetivo (y)
X = df[['Precio', 'Media']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.2, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Input(shape=(X_train.shape[1],)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [24]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)


In [25]:
# Hacer predicciones para puntos y precio
#puntos_predictions = model_puntos.predict(X_test_scaled)
#precio_predictions = model_precio.predict(X_test_scaled)

# Obtener el jugador con más puntos y el jugador con el precio más alto
#indice_max_puntos = puntos_predictions.argmax()
#indice_max_precio = precio_predictions.argmax()

#jugador_mas_puntos = df.loc[indice_max_puntos, 'Nombre']
#jugador_mas_precio = df.loc[indice_max_precio, 'Nombre']

#print(f"Jugador con más puntos predichos: {jugador_mas_puntos}")
#print(f"Jugador con precio más alto predicho: {jugador_mas_precio}")


In [26]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Obtener los 10 jugadores con más puntos predichos
top_10_jugadores_puntos = df.nlargest(10, 'Total_puntos')

# Obtener los 10 jugadores con los precios más altos predichos
top_10_jugadores_precio = df.nlargest(10, 'Precio')

# Impresion de los top 10 Jugadores con mayores puntos y precios 
print("Top 10 jugadores con más puntos predichos:")
print(top_10_jugadores_puntos[['Nombre', 'Total_puntos']])

print("\nTop 10 jugadores con precios más altos predichos:")
print(top_10_jugadores_precio[['Nombre', 'Precio']])



4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/stepWARNING:tensorflow:5 out of the last 17 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x3083ceee0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Top 10 jugadores con más puntos predichos:
          Nombre  Total_puntos
0  J. Bellingham           173
1        T. Kubo           147
2   A. Griezmann           144


In [27]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step
MSE para puntos: 388.37
R^2 para puntos: 0.69
MSE para precio: 58077826879423.29
R^2 para precio: -0.59


In [28]:
# Crear un DataFrame con las predicciones
# Obtener los 10 jugadores con las predicciones más altas de puntos
top_10_puntos_predictions = pd.DataFrame({
    'Nombre': df.iloc[X_test.index]['Nombre'],  # Recuperar los nombres de los jugadores
    'Posicion': df.iloc[X_test.index]['posicion'],   # Utilizar la posición de los jugadores
    'Predicciones_Puntos': puntos_predictions.flatten(),  # Aplanar el array de predicciones
    'Predicciones_Precio': precio_predictions.flatten()  # Aplanar el array de predicciones
})

# Ordenar el DataFrame por Predicciones_Puntos en orden descendente
top_10_puntos_predictions = top_10_puntos_predictions.sort_values(by='Predicciones_Puntos', ascending=False)

# Tomar solo las primeras 10 filas (los 10 jugadores con las predicciones más altas)
top_10_puntos_predictions = top_10_puntos_predictions.head(10)

# Cambiar el nombre de la columna 'Predicciones_Puntos' a 'Puntos'
top_10_puntos_predictions = top_10_puntos_predictions.rename(columns={'Predicciones_Puntos': 'Puntos', 'Predicciones_Precio': 'Precio'})

# Guardar el DataFrame en un archivo CSV
top_10_puntos_predictions.to_csv('top_10_predicciones_puntos.csv', index=False)


In [29]:
top_10_puntos_predictions

,Nombre,Posicion,Puntos,Precio
0,J. Bellingham,mediocentro,189.221313,3781.593750
2,A. Griezmann,delantero,153.410156,3126.486572
399,M. Guiu,delantero,148.611725,2272.332275
10,Isco,mediocentro,130.884979,2619.285400
101,F. De Jong,mediocentro,121.836952,2403.360107
22,G. Moreno,delantero,120.425453,2398.980469
11,Sávio,delantero,119.647682,2376.213623
9,A. Budimir,delantero,117.139397,2255.192627
18,F. Valverde,mediocentro,115.941048,2325.386719
72,V. Muriqi,delantero,111.182152,2050.604492


In [30]:
# Guardar las predicciones en un archivo CSV
#predicciones_df.to_csv('predicciones.csv', index=False)

In [31]:
#joblib.dump(model_puntos, 'redes_neuronales.pkl')

In [32]:
####